In [4]:
from pathlib import Path
import numpy as np

In [73]:
def read(prefix='data', suffix='24', pad_width=None):
    
    lines = Path(f'{prefix}/{suffix}.txt').read_text().rstrip().split('\n')
    M = {}
    for t in ['#', '<', '^', '>', 'v']:
        M[t] = np.array([[(c == t) for c in l] for l in lines])

    return np.stack(list(M.values()))

def disp(M):
    print(*[''.join('#' if c else '.' for c in r) for r in M], sep='\n')

In [185]:
def moves(M,y,x):
    out = set()
    for (dy,dx) in [(0,1),(1,0),(-1,0),(0,-1), (0,0)]:
        ny, nx = y+dy, x+dx
        if (ny >= 0) and (ny < M.shape[1]) and (nx >= 0) and (nx < M.shape[2]):
            if M[:,ny,nx].sum() == 0:
                out.add((ny,nx))
    return out


In [186]:
def update(M):
    A = M[:,1:-1,1:-1] 
    A[1] = np.roll(A[1],shift=-1,axis=1)
    A[2] = np.roll(A[2],shift=-1,axis=0)
    A[3] = np.roll(A[3],shift=1,axis=1)
    A[4] = np.roll(A[4],shift=1,axis=0)

In [187]:
def find(M, ys=0,xs=1, yt=None, xt=None, start_steps=0, num_steps=100):
    M = M.copy()
    if yt is None:
        yt = M.shape[1] - 1
    if xt is None:
        xt = M.shape[2] - 2 

    states = {(ys,xs)}

    for _ in range(start_steps):
        update(M)

    for step in range(1,num_steps):
        update(M)
        states_nw = set()
        for (y,x) in states:
            states_nw = states_nw | moves(M,y,x)
        states = states_nw
        
        if (yt,xt) in states:
            return step

    return None
        

In [188]:
find(read('test'), num_steps=1000)

18

In [189]:
find(read(), num_steps=1000)

253

In [194]:
def find2(M, num_steps=100):
    ys, xs = 0, 1
    yt, xt = M.shape[1] - 1, M.shape[2] - 2
    tot = find(M, num_steps=num_steps)
    tot += find(M, ys=yt, xs=xt, yt=ys,xt=xs, num_steps=num_steps, start_steps=tot)
    tot += find(M, num_steps=num_steps, start_steps=tot)

    return tot 

In [195]:
find2(read('test'), num_steps=1000)

54

In [196]:
find2(read(), num_steps=1000)

794